# model preparation

In [21]:
import pandas as pd
import numpy as np

In [22]:
model_df = pd.read_csv('feature_df2.csv')
model_df['date_time'] = pd.to_datetime(model_df['date_time'])
model_df['real_time'] = pd.to_datetime(model_df['real_time'])
model_df.set_index('date_time', inplace=True)

In [23]:
train_start = '2021-01-01'
train_end = '2024-01-01'
test_start = '2024-01-01'
test_end = '2024-06-01'

train_df = model_df[(model_df.index >= train_start) & (model_df.index < train_end)]
test_df = model_df[(model_df.index >= test_start) & (model_df.index < test_end)]

In [24]:
Y = train_df.columns[12:19].tolist()
X = train_df.columns[8:11].tolist() + train_df.columns[19:].tolist()

    
Y_train_vars = globals()
for i, col_name in enumerate(Y):
    Y_train_vars[f'Y_train_{i}'] = train_df[[col_name]]

X_train_vars = globals()
for i, col_name in enumerate(X):
    X_train_vars[f'X_train_{i}'] = train_df[[col_name]]
    

Y_test = test_df.columns[12:19].tolist()
X_test = test_df.columns[8:11].tolist() + train_df.columns[19:].tolist()
Y_test_vars = globals()
for i, col_name in enumerate(Y_test):
    Y_test_vars[f'Y_test_{i}'] = test_df[[col_name]]
X_test_vars = globals()
for i, col_name in enumerate(X_test):
    X_test_vars[f'X_test_{i}'] = test_df[[col_name]]


In [25]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

In [26]:
def train_and_test(X_train, Y_train, X_test, Y_test):
    
    # 初始化模型
    model = LinearRegression()
    
    # 用于存储历史损失和准确率
    train_losses, test_losses = [], []
    train_scores, test_scores = [], []
    
    # 最佳参数和损失
    best_params = None
    lowest_loss = np.inf

    model.fit(X_train, Y_train)
        
    # 在训练集和测试集上做预测
    Y_train_pred = model.predict(X_train)
    Y_test_pred = model.predict(X_test)
    
    # 计算损失
    train_loss = mean_squared_error(Y_train, Y_train_pred)
    test_loss = mean_squared_error(Y_test, Y_test_pred)
    train_losses.append(train_loss)
    test_losses.append(test_loss)
        
    # 计算R2分数
    train_score = r2_score(Y_train, Y_train_pred)
    test_score = r2_score(Y_test, Y_test_pred)
    train_scores.append(train_score)
    test_scores.append(test_score)
    
    lowest_loss = test_loss
    coef_params = model.coef_
    r2_scores = test_scores

    return coef_params, lowest_loss, r2_scores, Y_train_pred, Y_test_pred

# coef_params, lowest_loss, r2_scores, Y_train_pred, Y_test_pred = train_and_test(X_train_159, Y_train_1, X_test_159, Y_test_1)

In [27]:
results = []

for i, y_col in enumerate(Y):
    Y_train = Y_train_vars[f'Y_train_{i}']
    Y_test = Y_test_vars[f'Y_test_{i}']
    for j, x_col in enumerate(X):
        X_train = X_train_vars[f'X_train_{j}']
        X_test = X_test_vars[f'X_test_{j}']
    
        coef_params, lowest_loss, r2_scores, Y_train_pred, Y_test_pred = train_and_test(X_train, Y_train, X_test, Y_test)
        test_df[f'{x_col}_pred_rtn'] = Y_test_pred
        
        # print(f"Results for {f'Y_train_{i}', f'Y_test_{i}', f'X_train_{j}', f'X_test_{j}'}:")
        # print("Best Parameters:", coef_params)
        # print("Lowest Test Loss:", lowest_loss)
        # print("R2 Scores:", r2_scores)
        # print("-" * 40)
        
        results.append((Y_train.columns[0], X_train.columns[0], lowest_loss, r2_scores, coef_params))
        results_df = pd.DataFrame(results, columns=['Y', 'X', 'Lowest_Loss', 'R2_Scores', 'Coef_params'])
        result_df = results_df.sort_values(by='Lowest_Loss').head(10)

result_df

C:\Users\Young\AppData\Local\Temp\ipykernel_5048\1690144614.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[f'{x_col}_pred_rtn'] = Y_test_pred
C:\Users\Young\AppData\Local\Temp\ipykernel_5048\1690144614.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[f'{x_col}_pred_rtn'] = Y_test_pred
C:\Users\Young\AppData\Local\Temp\ipykernel_5048\1690144614.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

,Y,X,Lowest_Loss,R2_Scores,Coef_params
158,return_1h,base_asset_trades_volume_ratio,0.000024,[0.31270531699231296],[[0.024840422185711735]]
70,return_1h,Diff_SMA_4h,0.000034,[0.004699179973164602],[[-0.0014404329683758354]]
194,return_1h,quote_asset_Quote_change_14d,0.000034,[0.003915247918054421],[[0.001091543482966284]]
142,return_1h,base_asset_Quote_change_14d,0.000034,[0.003243338462526335],[[0.0010162533702493142]]
18,return_1h,qav_Diff_SMA_4h,0.000034,[0.003165276055131061],[[-0.0012078945629763191]]
193,return_1h,quote_asset_Quote_change_7d,0.000034,[0.002359019715722166],[[0.000881713004022851]]
71,return_1h,Diff_SMA_8h,0.000034,[0.002357553739298912],[[-0.0009786867294646442]]
93,return_1h,TVMI_8h,0.000034,[0.0021970136585599542],[[-0.0010770702251154819]]
195,return_1h,quote_asset_TVMI_1h,0.000034,[0.0020654843978321624],[[0.0018750118486470465]]
141,return_1h,base_asset_Quote_change_7d,0.000034,[0.0020397691215370273],[[0.0007757321148548265]]


In [29]:
test_df

,real_time,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,...,quote_asset_TVMI_14d_pred_rtn,quote_asset_TVMO_1h_pred_rtn,quote_asset_TVMO_4h_pred_rtn,quote_asset_TVMO_8h_pred_rtn,quote_asset_TVMO_1d_pred_rtn,quote_asset_TVMO_3d_pred_rtn,quote_asset_TVMO_7d_pred_rtn,quote_asset_TVMO_14d_pred_rtn,quote_asset_trade_close_ratio_pred_rtn,quote_asset_trades_volume_ratio_pred_rtn
date_time,,,,,,,,,,,,,,,,,,,,,
2024-01-01 00:01:00,2024-01-01 00:00:00,1704067200000,42314.0,42603.2,42289.6,42503.5,8459.477,1.704071e+12,0.239906,0.240048,...,0.010876,0.011182,0.010938,0.010778,0.010836,0.011047,0.011059,0.010813,0.019011,0.035551
2024-01-01 01:01:00,2024-01-01 01:00:00,1704070800000,42503.5,42832.0,42462.0,42647.9,9043.411,1.704074e+12,0.257947,0.245715,...,0.010976,0.010780,0.011058,0.010798,0.010835,0.011103,0.011215,0.010936,0.018759,0.031800
2024-01-01 02:01:00,2024-01-01 02:00:00,1704074400000,42647.9,42676.9,42530.0,42620.4,4653.067,1.704078e+12,0.131437,0.142390,...,0.008977,0.011160,0.011199,0.011205,0.011154,0.010414,0.007536,0.008510,0.025141,0.020062
2024-01-01 03:01:00,2024-01-01 03:00:00,1704078000000,42620.5,42630.0,42270.0,42369.8,8119.880,1.704082e+12,0.229951,0.229298,...,0.009997,0.010780,0.011019,0.011025,0.010994,0.010779,0.009396,0.009750,0.021853,0.014853
2024-01-01 04:01:00,2024-01-01 04:00:00,1704081600000,42369.8,42439.8,42235.2,42436.6,6356.536,1.704085e+12,0.179229,0.184692,...,0.009734,0.011194,0.011015,0.011092,0.011040,0.010684,0.008901,0.009432,0.022709,0.025663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-31 19:01:00,2024-05-31 19:00:00,1717182000000,67456.2,68059.4,67388.2,67719.9,12522.881,1.717186e+12,0.569703,0.397100,...,0.014276,0.010780,0.010962,0.011024,0.010694,0.012194,0.017304,0.013963,0.013533,0.091149
2024-05-31 20:01:00,2024-05-31 20:00:00,1717185600000,67720.0,67834.4,67633.1,67755.0,3592.345,1.717189e+12,0.161819,0.156664,...,0.009049,0.011156,0.011106,0.011217,0.011188,0.010300,0.008724,0.009359,0.026201,0.069297
2024-05-31 21:01:00,2024-05-31 21:00:00,1717189200000,67755.0,67763.0,67557.7,67602.3,2543.573,1.717193e+12,0.113826,0.114542,...,0.008438,0.011166,0.011087,0.011191,0.011247,0.010080,0.007711,0.008820,0.027675,0.056975


In [30]:
pred_rtn_columns = [x + '_pred_rtn' for x in result_df['X'].unique()]
y_column = result_df['Y'].iloc[0]
columns_to_keep = [y_column] + pred_rtn_columns
model_df = test_df[columns_to_keep]
model_df.to_csv('model_out.csv')
model_df.head()

,return_1h,base_asset_trades_volume_ratio_pred_rtn,Diff_SMA_4h_pred_rtn,quote_asset_Quote_change_14d_pred_rtn,base_asset_Quote_change_14d_pred_rtn,qav_Diff_SMA_4h_pred_rtn,quote_asset_Quote_change_7d_pred_rtn,Diff_SMA_8h_pred_rtn,TVMI_8h_pred_rtn,quote_asset_TVMI_1h_pred_rtn,base_asset_Quote_change_7d_pred_rtn
date_time,,,,,,,,,,,
2024-01-01 00:01:00,0.004478,0.005943,0.010990,0.007695,0.019037,0.011010,0.008873,0.011075,0.010726,0.011022,0.015736
2024-01-01 01:01:00,0.003397,0.008214,0.010987,0.006305,0.022306,0.011141,0.011756,0.011070,0.010755,0.010979,0.009153
2024-01-01 02:01:00,-0.000645,0.014419,0.010981,0.005184,0.024851,0.011634,0.008773,0.010910,0.011400,0.011248,0.015970
2024-01-01 03:01:00,-0.005880,0.017034,0.010992,0.007020,0.020645,0.010816,0.012094,0.011024,0.010944,0.010825,0.008274
2024-01-01 04:01:00,0.001577,0.011152,0.010989,0.008587,0.017029,0.011051,0.012367,0.010949,0.011225,0.011023,0.007608
